In [49]:
import pandas as pd
import nltk
import gensim
from gensim import downloader
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from scipy.spatial import distance
import re
import numpy as np

df_books = pd.read_csv("Books.csv")

/var/folders/v2/rwg0t7796jsbrq_0j88y5mk40000gn/T/ipykernel_26662/697850235.py:11: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books = pd.read_csv("Books.csv")


In [2]:
df_summary = pd.read_csv('booksummaries/booksummaries.txt', sep = '\t')
df_summary.columns = ['Code1', 'Code2', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Code3', 'Book-Summary']
df_summary

,Code1,Code2,Book-Title,Book-Author,Year-Of-Publication,Code3,Book-Summary
0,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
1,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
2,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...
3,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...
4,2152,/m/0x5g,All Quiet on the Western Front,Erich Maria Remarque,1929-01-29,"{""/m/098tmk"": ""War novel"", ""/m/016lj8"": ""Roman...","The book tells the story of Paul Bäumer, a Ge..."
...,...,...,...,...,...,...,...
16553,36934824,/m/0m0p0hr,Under Wildwood,Colin Meloy,2012-09-25,NaN,"Prue McKeel, having rescued her brother from ..."
16554,37054020,/m/04f1nbs,Transfer of Power,Vince Flynn,2000-06-01,"{""/m/01jfsb"": ""Thriller"", ""/m/02xlf"": ""Fiction""}",The reader first meets Rapp while he is doing...
16555,37122323,/m/0n5236t,Decoded,Jay-Z,2010-11-16,"{""/m/0xdf"": ""Autobiography""}",The book follows very rough chronological ord...
16556,37132319,/m/0n4bqb1,America Again: Re-becoming The Greatness We Ne...,Stephen Colbert,2012-10-02,NaN,Colbert addresses topics including Wall Stree...


In [3]:
df_books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [38]:
df_merged = pd.merge(df_summary, df_books, how='inner', on =['Book-Title'])
df_merged = df_merged.drop(['Code1','Code2','Code3','Publisher', 'Year-Of-Publication_x', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L', 'Book-Author_y', 'Year-Of-Publication_y'], axis=1)
df_merged = df_merged[df_merged['Book-Title'] != 'Deathstalker Rebellion']
df_merged

,Book-Title,Book-Author_x,Book-Summary,ISBN
0,A Clockwork Orange,Anthony Burgess,"Alex, a teenager living in near-future Englan...",345316413X
1,A Clockwork Orange,Anthony Burgess,"Alex, a teenager living in near-future Englan...",0749318473
2,A Clockwork Orange,Anthony Burgess,"Alex, a teenager living in near-future Englan...",0345256328
3,A Clockwork Orange,Anthony Burgess,"Alex, a teenager living in near-future Englan...",0345273214
4,A Clockwork Orange,Anthony Burgess,"Alex, a teenager living in near-future Englan...",3453130790
...,...,...,...,...
13783,Remote Control,Andy McNab,The series follows the character of Nick Ston...,0451191692
13784,Remote Control,Andy McNab,The series follows the character of Nick Ston...,0345254457
13785,The Simpsons: A Complete Guide to Our Favorite...,Matt Groening,"{| class=""wikitable"" |- !Seasons covered !Boo...",0060952520
13786,Transfer of Power,Vince Flynn,The reader first meets Rapp while he is doing...,0671023209


In [5]:
len(df_merged['Book-Title'].unique())

4202

In [50]:
glove_vectors = gensim.downloader.load('word2vec-google-news-300')

In [7]:
df_ratings = pd.read_csv('Ratings.csv')
df_ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [8]:
stop_words = set(stopwords.words('english'))

In [52]:
glove_vectors['fire']

array([ 3.55468750e-01,  1.83593750e-01,  1.49414062e-01, -9.37500000e-02,
        1.78710938e-01, -8.39843750e-02, -6.39648438e-02, -2.57812500e-01,
       -1.17187500e-01,  1.36718750e-01,  2.27539062e-01, -2.55859375e-01,
       -1.85546875e-01, -2.08007812e-01, -1.70898438e-01,  2.56347656e-02,
       -1.20117188e-01, -1.08398438e-01, -6.34765625e-02,  1.51977539e-02,
        1.50390625e-01, -2.05078125e-01,  2.02148438e-01, -5.51757812e-02,
        1.75781250e-02,  1.55273438e-01,  2.42919922e-02,  1.37695312e-01,
        3.22265625e-01, -1.06445312e-01, -6.49414062e-02, -3.84765625e-01,
       -3.04687500e-01, -2.38281250e-01, -1.69921875e-01, -1.78710938e-01,
       -1.34765625e-01,  1.03027344e-01, -1.31835938e-01,  2.35595703e-02,
        4.51660156e-02, -1.52343750e-01,  1.43554688e-01, -2.57812500e-01,
       -8.59375000e-02, -2.57812500e-01, -1.60156250e-01, -1.34765625e-01,
       -1.41601562e-01,  1.92382812e-01,  2.81250000e-01,  1.43554688e-01,
        2.08007812e-01, -

In [54]:
title_dict = {}
summary_dict = {}

for row in df_merged.index:
    title = df_merged['Book-Title'][row]
    title_nonum = re.sub(r'\d+', '', title) #remove number
    tokenizer = RegexpTokenizer(r'\w+') #remove punctuation
    title_tokens = tokenizer.tokenize(title_nonum) 
    filtered_title_tokens = [w for w in title_tokens if not w.lower() in stop_words] #remove stop words

    summary = df_merged['Book-Summary'][row]
    summary_nonum = re.sub(r'\d+', '', summary) #remove number
    tokenizer = RegexpTokenizer(r'\w+') #remove punctuation
    summary_tokens = tokenizer.tokenize(summary_nonum) 
    filtered_summary_tokens = [w for w in summary_tokens if not w.lower() in stop_words] #remove stop words

    title_vectors = np.zeros(300)
    summary_vectors = np.zeros(300)

    n = 0
    for title_token in filtered_title_tokens:
        try:
            title_token_vec = glove_vectors[title_token]
        except:
            title_token_vec = np.zeros(300)
        title_vectors += title_token_vec
        n += 1
    title_dict[title] = title_vectors / n
    
    n = 0
    for summary_token in filtered_summary_tokens:
        try:
            summary_token_vec = glove_vectors[summary_token]
        except:
            summary_token_vec = np.zeros(300)
        summary_vectors += summary_token_vec
        n += 1
    summary_dict[title] = summary_vectors / n

    #print(title, summary_vectors / n)
    
print(len(summary_dict))

/var/folders/v2/rwg0t7796jsbrq_0j88y5mk40000gn/T/ipykernel_26662/2632404893.py:28: RuntimeWarning: invalid value encountered in divide
  title_dict[title] = title_vectors / n


5850


In [55]:
v1 = title_dict['The Hobbit']
v2 = title_dict['A Clockwork Orange']
dst = distance.euclidean(v1, v2)
dst


4.562087672492212

In [56]:
v1 = summary_dict['The Hobbit']
v2 = summary_dict['Children of Dune']
dst = distance.euclidean(v1, v2)
dst

0.6010977627961166

In [62]:
books_count = len(summary_dict)
books_distance = np.zeros(shape=(books_count, books_count))
print(books_count)

i = 0
for key1 in summary_dict:
    j = 0
    summary_vec1 = summary_dict[key1]
    for key2 in summary_dict:
        summary_vec2 = summary_dict[key2]
        dst = distance.euclidean(summary_vec1, summary_vec2)
        books_distance[i][j] = dst
        j+=1
        #print(key1, key2, dst)
        #print(j)
    if i % 500 == 0:
        print(i)
    
    i+=1

books_distance

5850
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500


array([[0.        , 0.37544038, 0.55792849, ..., 0.64643876, 1.35175962,
        0.55920757],
       [0.37544038, 0.        , 0.4911016 , ..., 0.65173414, 1.39196772,
        0.52858901],
       [0.55792849, 0.4911016 , 0.        , ..., 0.68704658, 1.38754327,
        0.66806606],
       ...,
       [0.64643876, 0.65173414, 0.68704658, ..., 0.        , 1.42089039,
        0.64450647],
       [1.35175962, 1.39196772, 1.38754327, ..., 1.42089039, 0.        ,
        1.42476137],
       [0.55920757, 0.52858901, 0.66806606, ..., 0.64450647, 1.42476137,
        0.        ]])